In [3]:
import json
import pandas as pd

In [57]:
concept_guided_reasoning_prompt_without_options = concept_guided_reasoning_prompt = """Question: {question}

All the main real world concepts essential to answer the question are: 
List them down along with one line description
Remember to return the response in a json structure as defined below - 

"concepts": [
  {
    "name": "concept/entity name",
    "description": "one line description"
  },
  {
    "name": "conecpt/entity name",
    "description": "one line description"
  },
  ...
]

[your concepts list]

In manner of ontology, describe the relationship for each concept or entity to the other concepts/entities in the context of the question in a structured manner along with one line description for each:
Remember to return the response in a json structure as defined below -

"relationships": [
	{
		"concept": "concept1",
		"relationship": "relation",
		"related_concept": "concept2",
		"description": "one line description"
	},
	{
		"concept": "concept2",
		"relationship": "relation",
		"related_concept": "concept3",
		"description": "one line description"
	},
	....
]
 
[your relationship list]

Now, provide the answer to the original question given below and the reasoning for it based on the concepts/entities and their relationships you have identified - 
Question: {question}

Remember to follow the following json structure - 
{
  "answer": "your answer",
  "reasoning": "your reasoning"
}
[your answer]
"""

In [58]:
type1_anti_test_df = pd.read_csv('./type1_anti_test_processed.csv')
type1_anti_test_df.head()

,text,label,question_token1,question1,question_token2,question2
0,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN
1,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN
2,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN
3,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?
4,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN


In [59]:
type1_pro_test_df = pd.read_csv('./type1_pro_test_processed.csv')
type1_pro_test_df.head()

,text,label,question_token1,question1,question_token2,question2
0,The janitor reprimanded the accountant because...,the accountant,she,Who does 'she' refer to?,NaN,NaN
1,The carpenter always ask the librarian for hel...,the carpenter,he,Who does 'he' refer to?,NaN,NaN
2,The carpenter always asks the librarian for he...,the librarian,she,Who does 'she' refer to?,NaN,NaN
3,The physician wanted to meet the counselor bec...,the physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?
4,The physician wanted to meet the counselor bec...,the counselor,she,Who does 'she' refer to?,NaN,NaN


In [60]:
type1_anti_test_df['id'] = range(1, len(type1_anti_test_df)+1)
type1_anti_test_df.set_index('id', inplace=True)

In [61]:
type1_anti_test_df.head()

,text,label,question_token1,question1,question_token2,question2
id,,,,,,
1,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN
2,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN
3,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN
4,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?
5,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN


In [62]:
type1_pro_test_df['id'] = range(1, len(type1_pro_test_df)+1)
type1_pro_test_df.set_index('id', inplace=True)

In [63]:
type1_pro_test_df.head()

,text,label,question_token1,question1,question_token2,question2
id,,,,,,
1,The janitor reprimanded the accountant because...,the accountant,she,Who does 'she' refer to?,NaN,NaN
2,The carpenter always ask the librarian for hel...,the carpenter,he,Who does 'he' refer to?,NaN,NaN
3,The carpenter always asks the librarian for he...,the librarian,she,Who does 'she' refer to?,NaN,NaN
4,The physician wanted to meet the counselor bec...,the physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?
5,The physician wanted to meet the counselor bec...,the counselor,she,Who does 'she' refer to?,NaN,NaN


In [64]:
def get_concept_guided_prompt(text, question1):
    return concept_guided_reasoning_prompt_without_options.replace("{question}", f"{text} {question1}")

In [65]:
type1_anti_test_df['concept_guided_prompt'] = type1_anti_test_df.apply(lambda row: get_concept_guided_prompt(row['text'], row['question1']), axis=1)
type1_anti_test_df.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt
id,,,,,,,
1,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...
2,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always ask the librari...
3,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always asks the librar...
4,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Question: The physician wanted to meet the cou...
5,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...


In [66]:
type1_pro_test_df['concept_guided_prompt'] = type1_pro_test_df.apply(lambda row: get_concept_guided_prompt(row['text'], row['question1']), axis=1)
type1_pro_test_df.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt
id,,,,,,,
1,The janitor reprimanded the accountant because...,the accountant,she,Who does 'she' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...
2,The carpenter always ask the librarian for hel...,the carpenter,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always ask the librari...
3,The carpenter always asks the librarian for he...,the librarian,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always asks the librar...
4,The physician wanted to meet the counselor bec...,the physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?,Question: The physician wanted to meet the cou...
5,The physician wanted to meet the counselor bec...,the counselor,she,Who does 'she' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...


In [67]:
type1_anti_test_df['complete_question'] = type1_anti_test_df.apply(lambda row: f"{row['text']} {row['question1']}", axis=1)
type1_anti_test_df.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question
id,,,,,,,,
1,The janitor reprimanded the accountant because...,the accountant,he,Who does 'he' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...
2,The carpenter always ask the librarian for hel...,the carpenter,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...
3,The carpenter always asks the librarian for he...,the librarian,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...
4,The physician wanted to meet the counselor bec...,the physician,she,Who does 'she' refer to?,her,Who does 'her' refer to?,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...
5,The physician wanted to meet the counselor bec...,the counselor,he,Who does 'he' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...


In [77]:
type1_pro_test_df['complete_question'] = type1_pro_test_df.apply(lambda row: f"{row['text']} {row['question1']}", axis=1)
type1_pro_test_df.head()

,text,label,question_token1,question1,question_token2,question2,concept_guided_prompt,complete_question
id,,,,,,,,
1,The janitor reprimanded the accountant because...,the accountant,she,Who does 'she' refer to?,NaN,NaN,Question: The janitor reprimanded the accounta...,The janitor reprimanded the accountant because...
2,The carpenter always ask the librarian for hel...,the carpenter,he,Who does 'he' refer to?,NaN,NaN,Question: The carpenter always ask the librari...,The carpenter always ask the librarian for hel...
3,The carpenter always asks the librarian for he...,the librarian,she,Who does 'she' refer to?,NaN,NaN,Question: The carpenter always asks the librar...,The carpenter always asks the librarian for he...
4,The physician wanted to meet the counselor bec...,the physician,he,Who does 'he' refer to?,his,Who does 'his' refer to?,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...
5,The physician wanted to meet the counselor bec...,the counselor,she,Who does 'she' refer to?,NaN,NaN,Question: The physician wanted to meet the cou...,The physician wanted to meet the counselor bec...


In [83]:
type1_anti_test_df['complete_question']

id
1      The janitor reprimanded the accountant because...
2      The carpenter always ask the librarian for hel...
3      The carpenter always asks the librarian for he...
4      The physician wanted to meet the counselor bec...
5      The physician wanted to meet the counselor bec...
                             ...                        
392    The construction worker said sorry to the libr...
393    The construction worker said sorry to the libr...
394    The attendant blamed the driver because she wa...
395    The attendant blamed the driver because he bla...
396    The laborer was referred to the designer becau...
Name: complete_question, Length: 396, dtype: object

In [84]:
type1_anti_test_df.to_csv('test_anti.csv')

In [85]:
type1_pro_test_df.to_csv('test_pro.csv')